In [8]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
from keras.models import model_from_json
import os

In [13]:
batch_size = 64
num_classes = 21

# input image dimensions
img_rows, img_cols = 45, 45

# the data, split between train and test sets
x_train = np.load("saved_data/training_data.npy")
y_train = np.load("saved_data/training_labels.npy")

x_test = np.load("saved_data/test_data.npy")
y_test = np.load("saved_data/test_labels.npy")
print(x_test.shape)
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255



(38074, 45, 45)


In [17]:
# convert class vectors to binary class matrices
y_tr = keras.utils.to_categorical(y_train, num_classes)
y_te = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_tr,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_te))
score = model.evaluate(x_test, y_te, verbose=0)

Train on 152343 samples, validate on 38074 samples
Epoch 1/1
152320/152343 [============================>.] - ETA: 0s - loss: 0.3930 - acc: 0.8939

In [5]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")